# Check if the keypoints are duplicated

\\wsl.localhost\Ubuntu\home\pkhamchuai\codes\spppt-2\output\SIFT\SIFT_20100_0.0001_0_1_1_20240705-150230_BFMatcher_RANSAC_test

In [29]:
import argparse
import glob
import numpy as np
import os
import time
import cv2
import torch
import matplotlib.pyplot as plt
import random
import math
from skimage.metrics import structural_similarity as ssim
from skimage.measure import ransac
from skimage.transform import FundamentalMatrixTransform, AffineTransform
# Suppress the specific warning
import warnings
import csv
import sys
from IPython.utils.capture import capture_output
from datetime import datetime

# Stub to warn about opencv version.
if int(cv2.__version__[0]) < 3: # pragma: no cover
  print('Warning: OpenCV 3 is not installed')

# Jet colormap for visualization.
myjet = np.array([[0.        , 0.        , 0.5       ],
                  [0.        , 0.        , 0.99910873],
                  [0.        , 0.37843137, 1.        ],
                  [0.        , 0.83333333, 1.        ],
                  [0.30044276, 1.        , 0.66729918],
                  [0.66729918, 1.        , 0.30044276],
                  [1.        , 0.90123457, 0.        ],
                  [1.        , 0.48002905, 0.        ],
                  [0.99910873, 0.07334786, 0.        ],
                  [0.5       , 0.        , 0.        ]])

from utils.SuperPoint import SuperPointFrontend, PointTracker, load_image
from utils.datagen import datagen
from utils.utils0 import *
from utils.utils1 import *
from utils.utils1 import ModelParams, print_summary, DL_affine_plot

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f'Device: {device}')

image_size = 256

def process_image(image):
    # squeeze dimensions 0 and 1
    image = image.squeeze(0).squeeze(0)
    # convert to numpy array
    image = image.cpu().numpy()
    # normalize image to range 0 to 1
    image = (image/np.max(image)).astype('float32')
    return image


Device: cuda


In [30]:
def run(model_params, method1='BFMatcher', method2='RANSAC', plot=1):
    
    test_dataset = datagen(model_params.dataset, False, model_params.sup)

    # Create output directory
    output_dir = f"output/{args.model}_{model_params.get_model_code()}_{timestamp}_{method1}_{method2}_test"
    os.makedirs(output_dir, exist_ok=True)

    metrics = []
    # create a csv file to store the metrics
    csv_file = f"{output_dir}/metrics.csv"

    num_failed = 0

    testbar = tqdm(test_dataset, desc=f'Testing:')
    for i, data in enumerate(testbar, 0):
        if i != 90:
            continue
        elif i == 90:
            # Get images and affine parameters
            source_image, target_image, affine_params_true, points1, points2, points1_2_true = data
            points1 = points1.squeeze(0).cpu().numpy()
            points2 = points2.squeeze(0).cpu().numpy()
                
            # process images
            source_image = process_image(source_image)
            target_image = process_image(target_image)
            # print(f"source_image: {source_image.shape}")
            # print(f"target_image: {target_image.shape}")

            # Extract keypoints and descriptors using SIFT
            sift = cv2.SIFT_create()
            kp1, desc1 = sift.detectAndCompute(source_image, None)
            kp2, desc2 = sift.detectAndCompute(target_image, None)

            # print(desc1.shape)
            # print(desc2.shape)

            # pad the smaller desc to the same size with zeros
            # if desc1.shape[0] < desc2.shape[0]:
            #     desc1 = np.pad(desc1, ((0, desc2.shape[0] - desc1.shape[0]), (0, 0)), mode='constant')
            #     kp1 = kp1 + tuple([cv2.KeyPoint(x=kp1[0].pt[0], y=kp1[0].pt[1], size=0)])
            # elif desc1.shape[0] > desc2.shape[0]:
            #     desc2 = np.pad(desc2, ((0, desc1.shape[0] - desc2.shape[0]), (0, 0)), mode='constant')
            #     kp2 = kp2 + tuple([cv2.KeyPoint(x=kp2[0].pt[0], y=kp2[0].pt[1], size=0)])

            if method1 == 'BFMatcher':
                # Match keypoints using nearest neighbor search
                bf = cv2.BFMatcher()
                matches = bf.knnMatch(desc1, desc2, k=2)

                good = []
                try:
                    for m,n in matches:
                        if m.distance < 0.75*n.distance:
                            good.append([m])
                except ValueError:
                    good = []

                # img3 = cv2.drawMatchesKnn(source_image, kp1, target_image, kp2,good,None,flags=cv2.DrawMatchesFlags_NOT_DRAW_SINGLE_POINTS)
                # plt.imshow(img3), plt.show()

                matches = np.array([m for m in matches])
                matches1 = np.float32([kp1[m[0].queryIdx].pt for m in good]).reshape(-1, 2)
                matches2 = np.float32([kp2[m[0].trainIdx].pt for m in good]).reshape(-1, 2)
                # print(f"matches1: {matches1}")
                # print(f"matches2: {matches2}")

                # tracker = PointTracker(2, nn_thresh=0.7)
                # matches = tracker.ransac(desc1, desc2, matches)

                # print(f"pair: {i+1}, matches: {matches.shape}")

            elif method1 == 'FLANN':
                FLANN_INDEX_KDTREE = 1
                index_params = dict(algorithm = FLANN_INDEX_KDTREE, trees = 5)
                search_params = dict(checks = 50)
                
                flann = cv2.FlannBasedMatcher(index_params, search_params)
                
                matches = flann.knnMatch(desc1,desc2,k=2)

                # Apply ratio test to filter out ambiguous matches
                good_matches = []
                for m, n in matches:
                    if m.distance < 0.75 * n.distance:
                        good_matches.append(m)

                # Apply RANSAC to filter out outliers
                matches1 = np.float32([kp1[m.queryIdx].pt for m in good_matches]).reshape(-1, 2)
                matches2 = np.float32([kp2[m.trainIdx].pt for m in good_matches]).reshape(-1, 2)

                # # Need to draw only good matches, so create a mask
                # matchesMask = [[0,0] for i in range(len(matches))]
                
                # # ratio test as per Lowe's paper
                # for i,(m,n) in enumerate(matches):
                #     if m.distance < 0.*n.distance:
                #         matchesMask[i]=[1,0]

                # draw_params = dict(matchColor = (0,255,0),
                #     singlePointColor = (255,0,0),
                #     matchesMask = matchesMask,
                #     flags = cv2.DrawMatchesFlags_DEFAULT)
                
                # img3 = cv2.drawMatchesKnn(source_image, kp1, target_image, kp2, matches, None, **draw_params)
                # plt.imshow(img3), plt.show()
            
            try:
                # M, mask = cv2.findHomography(matches1, matches2, cv2.RANSAC, 5.0)
                # print(f"M: {M}")
                # affine_transform1 = M[:2, :]
                if method2 == 'RANSAC':
                    affine_transform1, _ = cv2.estimateAffinePartial2D(matches1, matches2, method=cv2.RANSAC)
                elif method2 == 'LMEDS':
                    affine_transform1, _ = cv2.estimateAffinePartial2D(matches1, matches2, method=cv2.LMEDS)
                points1_transformed = cv2.transform(points1[None, :, :], affine_transform1)
                # print(f"matches1_transformed: {matches1_transformed.shape}")
                try:
                    points1_transformed = points1_transformed[0]
                except TypeError:
                    pass
                # transform image 1 and 2 using the affine transform matrix
                transformed_source_affine = cv2.warpAffine(source_image, affine_transform1, (256, 256))
                text = "success"

                if i < 100 and plot == 1:
                    plot_ = 1
                elif i < 100 and plot == 2:
                    plot_ = 2
                    # do the bitwise not operation to get the inverse of the image
                    source_image = cv2.bitwise_not(source_image)
                    target_image = cv2.bitwise_not(target_image)
                    transformed_source_affine = cv2.bitwise_not(transformed_source_affine)
                else:
                    plot_ = 0

                try:
                    matches1_2 = cv2.transform(matches1[None, :, :], affine_transform1)[0]
                    matches1, matches2, matches1_2 = matches1.T, matches2.T, matches1_2.T
                except:
                    matches1, matches2, matches1_2 = [], [], []
                    text = "failed"
                    plot_ = plot

                # print(f"matcches1: {matches1.shape}")
                # print(f"matches2: {matches2.shape}")
                # print(f"matches1_2: {matches1_2.shape}")
                results = DL_affine_plot(f"test", output_dir,
                    f"{i}", text, source_image, target_image, \
                    transformed_source_affine, \
                    matches1, matches2, matches1_2, desc1, desc2,
                    affine_params_true=affine_params_true,
                    affine_params_predict=np.round(affine_transform1, 3), 
                    heatmap1=None, heatmap2=None, plot=plot_)

            except cv2.error:
                # print(f"Error: {i}")
                # break
                affine_transform1 = np.array([[[1, 0, 0], [0, 1, 0]]])
                points1_transformed = points1
                transformed_source_affine = source_image
                text = "failed"
                num_failed += 1
                # continue

                if i < 100 and plot == 1:
                    plot_ = 1
                elif i < 100 and plot == 2:
                    plot_ = 2
                    # do the bitwise not operation to get the inverse of the image
                    source_image = cv2.bitwise_not(source_image)
                    target_image = cv2.bitwise_not(target_image)
                    transformed_source_affine = cv2.bitwise_not(transformed_source_affine)
                else:
                    plot_ = 0

                try:
                    matches1_2 = cv2.transform(matches1[None, :, :], affine_transform1)[0]
                    matches1, matches2, matches1_2 = matches1.T, matches2.T, matches1_2.T
                except:
                    matches1, matches2, matches1_2 = [], [], []
                    text = "failed"
                    plot_ = plot

                results = DL_affine_plot(f"test", output_dir,
                    f"{i}", text, source_image, target_image, \
                    transformed_source_affine, \
                    matches1, matches2, matches1_2, desc1, desc2,
                    affine_params_true=affine_params_true,
                    affine_params_predict=np.round(affine_transform1, 3), 
                    heatmap1=None, heatmap2=None, plot=plot_)

            try:
                points1, points2, points1_transformed = points1.T, points2.T, points1_transformed.T
            except:
                points1, points2, points1_transformed = [], [], []

            # print(f"points1: {points1.shape}")
            # print(f"points2: {points2.shape}")
            # print(f"points1_transformed: {points1_transformed.shape}")
            
            results = DL_affine_plot(f"test", output_dir,
                    f"{i}", text, source_image, target_image, \
                    transformed_source_affine, \
                    points1, points2, points1_transformed, desc1, desc2, 
                    affine_params_true=affine_params_true,
                    affine_params_predict=np.round(affine_transform1, 3), 
                    heatmap1=None, heatmap2=None, plot=False)

            # calculate metrics
            # matches1_transformed = results[0]
            mse_before = results[1]
            mse12 = results[2]
            tre_before = results[3]
            tre12 = results[4]
            mse12_image_before = results[5]
            mse12_image = results[6]
            ssim12_image_before = results[7]
            ssim12_image = results[8]

    return source_image, target_image, transformed_source_affine, matches1, matches2, matches1_transformed
            





In [31]:
import argparse
import sys

# Create a list of command-line arguments
# sys.argv = ['python', 'run_SIFT.py', '--model', 'SIFT', 
#             '--sup', '0', '--dataset', str(i), '--plot', '1',
#             '--method1', 'BFMatcher', '--method2', 'RANSAC',
#             ]

class Args:
    def __init__(self):
        self.dataset = 2
        self.sup = 1
        self.image = 1
        self.loss_image = 0
        self.num_epochs = 1
        self.learning_rate = 1e-4
        self.decay_rate = 0.96
        self.model = 'SIFT'
        self.model_path = None
        self.plot = 1
        self.method = 'RANSAC'

args = Args()

model_params = ModelParams(dataset=args.dataset, sup=args.sup, image=args.image,
                           loss_image=args.loss_image, num_epochs=args.num_epochs,
                           learning_rate=args.learning_rate, decay_rate=args.decay_rate)
model_params.print_explanation()

timestamp = datetime.now().strftime("%Y%m%d-%H%M%S")

# run the function
source_image, target_image, transformed_source_affine, matches1, matches2, matches1_transformed = run(model_params, method2='RANSAC', plot=1)

# plot the images
fig, ax = plt.subplots(1, 3, figsize=(15, 5))
ax[0].imshow(source_image, cmap='gray')
ax[0].scatter(matches1[0], matches1[1], c='r', s=5)
ax[0].set_title('Source Image')
ax[1].imshow(target_image, cmap='gray')
ax[1].scatter(matches2[0], matches2[1], c='r', s=5)
ax[1].set_title('Target Image')
ax[2].imshow(transformed_source_affine, cmap='gray')
ax[2].scatter(matches1_transformed[0], matches1_transformed[1], c='r', s=5)
ax[2].set_title('Transformed Source Image')
plt.show()

Model name:  dataset2_sup1_image1_points0_loss_image0
Model code:  21100_0.0001_0_1_1
Model params:  {'dataset': 2, 'sup': 1, 'image': 1, 'points': 0, 'loss_image_case': 0, 'loss_image': MSELoss(), 'loss_affine': <utils.utils1.loss_affine object at 0x7f1acb35e9e0>, 'learning_rate': 0.0001, 'decay_rate': 0.96, 'start_epoch': 0, 'num_epochs': 1, 'batch_size': 1, 'model_name': 'dataset2_sup1_image1_points0_loss_image0'}

Model name:  dataset2_sup1_image1_points0_loss_image0
Model code:  21100_0.0001_0_1_1
Dataset used:  Synthetic eye scaling
Supervised or unsupervised model:  Supervised
Loss image type:  Image used
Points used:  Points not used
Loss function case:  0
Loss function for image:  MSELoss()
Loss function for affine:  <utils.utils1.loss_affine object at 0x7f1acb35e9e0>
Learning rate:  0.0001
Decay rate:  0.96
Start epoch:  0
Number of epochs:  1
Batch size:  1




Testing::   0%|          | 0/100 [00:00<?, ?it/s]

Testing::  90%|█████████ | 90/100 [00:00<00:00, 188.26it/s]


error: OpenCV(4.6.0) /io/opencv/modules/features2d/src/sift.dispatch.cpp:477: error: (-5:Bad argument) image is empty or has incorrect depth (!=CV_8U) in function 'detectAndCompute'


In [ ]:
len(matches1[0])

108